In [1]:
#Develop ticker coverage table
import pandas as pd

TIC = pd.read_csv(r'/Users/joezhou/Downloads/R_ALL_TICKER.csv',sep='|')

TIC_U = TIC.drop_duplicates(subset=['colummn']).rename(columns={'colummn': 'TickName'})[['TickName']]

TIC_U['Counter'] = 1

KeyError: Index(['colummn'], dtype='object')

In [2]:

INF = pd.read_excel (r'/Users/joezhou/Downloads/R_ALL_INFO.xlsx') 
INF_U = INF.drop_duplicates(subset=['symbol']).rename(columns={'symbol': 'TickName'})[['TickName']]
INF_U['IN_INFO']="In Info Extract"

PRICE = pd.read_csv (r'/Users/joezhou/Downloads/R_ALL_Prices.csv', sep='|')
PRICE_U = PRICE.drop_duplicates(subset=['TickName'])[['TickName']]
PRICE_U['IN_PRICE']="In Price Extract"

FIN = pd.read_csv (r'/Users/joezhou/Downloads/R_ALL_FIN.csv', sep='|')
FIN_U = FIN.drop_duplicates(subset=['TickName'])[['TickName']]
FIN_U['IN_FIN']="In Financial Extract"


In [3]:
# Join it back together
TIC_COVERAGE = TIC_U.merge(INF_U,how='left',on='TickName').merge(PRICE_U,how='left',on='TickName').merge(FIN_U,how='left',on='TickName').fillna('Missing')


Tbl_Coverage = TIC_COVERAGE.groupby(['IN_INFO','IN_PRICE','IN_FIN'])['Counter'].count().reset_index().sort_values(by=['Counter'],ascending=False)

Tbl_Coverage['Percentage'] = 100 * Tbl_Coverage['Counter']  / Tbl_Coverage['Counter'].sum()

Tbl_Coverage.head(27)
# To-do: add market cap data in here to quantify the proportion missing

,IN_INFO,IN_PRICE,IN_FIN,Counter,Percentage
0,In Info Extract,In Price Extract,In Financial Extract,1017,46.332574
2,In Info Extract,Missing,In Financial Extract,476,21.685649
3,In Info Extract,Missing,Missing,265,12.072893
1,In Info Extract,In Price Extract,Missing,264,12.027335
5,Missing,Missing,Missing,169,7.699317
4,Missing,Missing,In Financial Extract,4,0.182232


In [4]:
TIC_COVERAGE.to_excel('/Users/joezhou/Downloads/V_ALL_TICKER_COVERAGE.xlsx')  
